In [1]:
!pip install pyod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.1.2-py3-none-any.whl size=190289 sha256=a534565275c371b217e24e7c6d813b88bfd3ba194b742fcf42fbf950d9213ecf
  Stored in directory: /root/.cache/pip/wheels/81/1b/61/aa85b78c3c0c8871f4231e3f4a03bb23cecb7db829498380ee
Successfully built pyod


In [2]:
from google.colab import files
uploaded = files.upload()

Saving titanic.csv to titanic.csv


In [3]:
import pandas as pd
titanic = pd.read_csv('titanic.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Selecting only the columns Survived, Pclass, Fare, and Sex
partial_titanic = titanic[["Survived", "Pclass", "Fare", "Sex"]]

In [5]:
# Change the categorical value Sex to numeric values
partial_titanic["Sex"] = partial_titanic["Sex"].map({
"male": 0,
"female": 1
})

<ipython-input-5-4f994d51c243>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partial_titanic["Sex"] = partial_titanic["Sex"].map({


In [6]:
# Import the KNN
from pyod.models.knn import KNN

In [7]:
# Initiate a KNN model
KNN_model = KNN()

In [8]:
# Fit the model to the whole dataset
KNN_model.fit(partial_titanic)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [9]:
# Find the labels
outlier_labels = KNN_model.labels_
# Find the number of outliers
number_of_outliers = len(outlier_labels[outlier_labels == 1])
print(number_of_outliers)

88


In [10]:
# Initiate a KNN model
KNN_model = KNN(contamination=0.01)
# Fit the model to the whole dataset
KNN_model.fit(partial_titanic)
# Find the labels
outlier_labels = KNN_model.labels_
# Find the number of outliers
number_of_outliers = len(outlier_labels[outlier_labels == 1])
print(number_of_outliers)

9


In [11]:
# Finding the outlier passengers
outliers = partial_titanic.iloc[outlier_labels == 1]

In [12]:
# Average fare price
round(partial_titanic["Fare"].mean(), 3)

32.204

In [13]:
# Import the LOF
from pyod.models.lof import LOF
# Initiate a LOF model
LOF_model = LOF()
# Train the model on the Titanic data
LOF_model.fit(partial_titanic)

LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2)

In [14]:
# Represent Mrs. Watson as a data point
mrs_watson = [[0, 1, 1000, 1]]

In [15]:
outlier = LOF_model.predict(mrs_watson)
print(outlier)

[1]


### A value of 1 indicates that Mrs. Watson is an outlier. This should make us suspicious that the rumor regarding Mrs. Watson is false